Baue die Verbindung zur MongoDB auf:

In [ ]:
import pymongo
from pymongo import MongoClient
import pandas as pd

# after that
#we make connection to our MongoDB Database
user = "root" # - your login
password = "BigDataAnalysisDatabase" # - your password 
host = "eeriefoods.de" # - it's ip address , where database is located 
port = 27017 # standart port
database = 'BDA' # name of your database's

# make our connection
client = MongoClient(f'mongodb://{user}:{password}@{host}:{port}/') #url and port where located your Mong

# Use our database
db = client['BDA']

print("database connected")


Lade die historischen täglichen Ölpreise aus der Datenbank:

In [ ]:
for document in db["OilPricesDaily"].find():
    print("dokument aus DB geladen")
    print(document)

Ersetze fehlende Werte in historischen Öldaten durch Werte vom Vortag:

In [ ]:
import pandas as pd

# Variablendefinition
seperator = '    '
last_value = 0
corrected_Oildata = document['data']

print('Geänderte Datensätze:')
print('Datum      Alter Wert  Neuer Wert')

# ersetze fehlende Daten durch Wert am Vortag
for i in reversed(range(len(corrected_Oildata))):
    if corrected_Oildata[i]['value'] == '.':
        print (corrected_Oildata[i]['date'],seperator,corrected_Oildata[i]['value'],seperator,last_value)
        corrected_Oildata[i]['value'] = last_value
        
    last_value = corrected_Oildata[i]['value']


# erzeuge DataFrame für korrigierte Öldaten
dfOil = pd.DataFrame(corrected_Oildata)
dfOil['date'] = pd.to_datetime(dfOil['date'])
dfOil = dfOil.apply(pd.to_numeric)
dfOil['date'] = pd.to_datetime(dfOil['date'])

print('DataFrame:')
print(dfOil)

Lade historische Tankdaten aus Tankpreise_ADAC.csv:

In [ ]:
# lade Tankdaten aus CSV-Datei
gas_prices = pd.read_csv('Tankpreise_adac.csv',sep = ';')


# rechne alte Werte in DM in EUR um
i=0
while gas_prices['Datum'][i][6:] < '2002':
    gas_prices['Diesel'][i] = gas_prices['Diesel'][i] / 1.95583
    gas_prices['Benzin'][i] = gas_prices['Benzin'][i] / 1.95583
    i = i+1

# erzeuge DataFrame für Tankdaten
dfGas = pd.DataFrame(gas_prices)
dfGas['Datum'] = pd.to_datetime(dfGas['Datum'],dayfirst=True)

print('DataFrame:')
print(dfGas)

Zeichne Graph mit Ölpreisen und Tankpreisen seit 1987:

In [ ]:
import matplotlib.pyplot as plt

# Variablendefinition für plot
days = len(corrected_Oildata)
oil_date = dfOil['date'][::-1]
oil_price = dfOil['value'][::-1]

# Achsenbeschriftung für plot
plt.title("Rohölpreise/Tankpreise", size="x-large")
plt.ylabel("Preis")
plt.xlabel("Zeit")

# erzeuge plot
plt.plot(oil_date,oil_price,label='Brent: USD pro Barrel')
plt.plot(dfGas['Datum'],dfGas['Diesel'],label='Diesel: ct pro l')
plt.plot(dfGas['Datum'],dfGas['Benzin'],label='Benzin: ct pro l')

plt.legend()

plt.show()


Zeichne Graph mit Ölpreisen und Tankpreisen für die letzten x-Jahre:
Bearbeite hierfür die Variable "years"

In [ ]:
import matplotlib.pyplot as plt

# Gib die Anzhal der Jahre ein
years = 4   # max 36


# rechne Jahre in Werktage und Monate um
days = years * 261

if years <= 24:
    month = years * 12
else:
    month = 24*12+(years-24)


# spiegele Arrays für Öldaten
oil_date = dfOil['date'][::-1]
oil_price = dfOil['value'][::-1]

# Achsenbeschriftung für plot
plt.title("Rohölpreise/Benzinpreise", size="x-large")
plt.ylabel("USD pro Barrel")
plt.xlabel("Zeit")

# erzeuge plot
plt.plot(oil_date[-days:],oil_price[-days:],label='Brent: USD pro Barrel')
plt.plot(dfGas['Datum'][-month:],dfGas['Diesel'][-month:],label='Diesel: ct pro l')
plt.plot(dfGas['Datum'][-month:],dfGas['Benzin'][-month:],label='Benzin: ct pro l')

plt.legend()

plt.show()


Historische Öldaten werden nach Monaten gruppiert und es wird der Durchschnitt pro Monat gebildet.
Dies ist eine Vorarbeit für die Berechnung der Korrelation zwichen Ölpreisen und Tankpreisen.
Die Ölpreise müssen für die Berechnung der Korrelation monatsweise vorliegen, da die Tankpreise ebenfalls nur monatlich erfasst wurden.

In [ ]:
# gruppiere Öldaten nach Jahren/Monaten
groupedOil= dfOil.groupby(dfOil.date.dt.to_period("M"))

#bilde Durchschnittswert pro Monat
averageOil = groupedOil['value'].mean()
averageOil

Berechne den Korrelationkoeffizienten für Rohöl und Diesel seit 1987:

In [ ]:
import numpy as np
# Korrelation Diesel

# Anzahl der Jahre, welche für Korrelation berücksichtigt werden
years = 24      # max 24

month = years * 12

# berechne Korrelationskoeffizienten
coef = np.corrcoef(averageOil[-month:], gas_prices['Diesel'][-month:])[0,1]

print('Korrelationskoeffizient: ',coef)

Berechne den Korrelationkoeffizienten für Rohöl und Benzin seit 1987:

In [ ]:
# Korrelation Benzin

# Anzahl der Jahre, welche für Korrelation berücksichtigt werden
years = 24      # max 24

month = years * 12

# berechne Korrelationskoeffizienten
coef = np.corrcoef(averageOil[-month:], gas_prices['Benzin'][-month:])[0,1]

print('Korrelationskoeffizient: ',coef)

Stelle die Korrelationskoeffizienten für Rohöl/Diesel und für Rohöl/Benzin als zeitlichen Verlauf dar:

In [ ]:
import datetime

year = []
coefDiesel = []
coefGas = []
month = 12

today = datetime.datetime.now().date()
currentYear = int(today.strftime('%Y'))

# kummuliert
for i in range(currentYear,1999,-1):
    year.append(i)
    coefVar = np.corrcoef(averageOil[-month:], gas_prices['Benzin'][-month:])[0,1]
    coefGas.append(coefVar)
    coefVar = np.corrcoef(averageOil[-month:], gas_prices['Diesel'][-month:])[0,1]
    coefDiesel.append(coefVar)
    month = month + 12

#for i in range(2000,currentYear + 1):
#    coefVar = np.corrcoef(averageOil[month-12:month], gas_prices['Benzin'][month-12:month])[0,1]
#    year.append(i)
#    coef.append(coefVar)
#    month = month + 12


# Achsenbeschriftung für plot
plt.title("Korrelationsentwicklung", size="x-large")
plt.ylabel("Korrelationskoeffizient")
plt.xlabel("Zeit")

# erzeuge plot
plt.plot(year,coefGas,label='Rohöl/Benzin')
plt.plot(year,coefDiesel,label='Rohöl/Diesel')

plt.legend()

plt.show()
  


Lade die Tankdaten der Tankerkönig API aus der MongoDB:

In [ ]:
from datetime import datetime

avg_diesel = []
avg_e10 = []
date = []

for i in db["tankerkoenig"].find():
    date.append(datetime.utcfromtimestamp(i['time']).strftime('%Y-%m-%d %H:%M:%S'))
    avg_diesel.append(i['avg_diesel'])
    avg_e10.append(i['avg_e10'])

print (date)



Stelle die Tankdaten der Tankerkönig API graphisch dar:

In [ ]:
plt.title("E10 / Diesel Preise", size="x-large")
plt.ylabel("Preis in €")
plt.xlabel("Zeit")

plt.plot(date,avg_e10,label='E10')
plt.plot(date,avg_diesel,label='Diesel')
    
plt.legend()

plt.show()

Lade die 5-minütlich erfassten Rohöldaten aus der MongoDB:

In [ ]:
oil_date2 = []

for i in db["OilPrices"].find():
    print (i['Time Series (5min)'])
